# Calculate Moon Coordinates 

### <span style="color:red; font-family:Georgia;">Robert Cameron, June 2017</span>

Based on the equations at:
http://www.stargazing.net/kepler/moon2.html and http://www.stjarnhimlen.se/comp/ppcomp.html

In [1]:
from math import *

In [2]:
# The time scale in the formulae below is days. 
# Hours, minutes, seconds are expressed as fractions of a day. 
# Day 0.0 occurs at 2000 Jan 0.0 UT (or 1999 Dec 31, 0:00 UT). 
# This "day number" d0 is computed as follows (y=year, m=month, D=date, UT=UT in hours+decimals):

def d2000(y,m,d,h):
    return 367*y - 7*(y + (m+9)//12) // 4 + 275*m//9 + d - 730530 + h/24.0

In [3]:
year = 1999
mon = 12
day = 31
UT = 12.0     # UT = hours of the day
d0 = d2000(year,mon,day,UT)
d0

0.5

In [4]:
# Local Longitude (loclon) and Latitude (loclat) for Stanford University (in degrees):
# From the Prime Meridian at Greenwich: east longitude is positive, west is negative.

loclon = 122.17
loclat = 37.43

In [5]:
twopi = 2.0 * pi

In [6]:
#   the function below returns an angle (in radians) in the range 0 to two pi
def pang(x):
    while (x <0):
        x += twopi
    while (x > twopi):
        x -= twopi
    return x

In [7]:
#   the myatan2 function below returns an angle in the range 0 to two pi
#   the Python atan2 function returns an angle in the range -pi to +pi
#
def myatan2(y, x):
    return pang(atan2(y, x))

In [8]:
# Primary orbital elements are:

#    N = longitude of the ascending node
#    i = inclination to the ecliptic (plane of the Earth's orbit)
#    w = argument of perihelion
#    a = semi-major axis, or mean distance from Sun
#    e = eccentricity (0=circle, 0-1=ellipse, 1=parabola)
#    M = mean anomaly (0 at perihelion; increases uniformly with time)

# Primary orbital elements of the Sun:

N_s = 0.0
i_s = 0.0
w_s = radians(282.9404 + 4.70935E-5 * d0)
a_s = 1.000000   # distance to the Sun in AU
e_s = 0.016709 - 1.151E-9 * d0
M_s = radians(356.0470 + 0.9856002585 * d0)

# Primary orbital elements of the Moon:

N_m = radians(125.1228 - 0.0529538083 * d0)
i_m = radians(5.1454)
w_m = radians(318.0634 + 0.1643573223 * d0)
a_m = 60.2666   # distance to the Moon in Earth radii
e_m = 0.054900
M_m = radians(115.3654 + 13.0649929509 * d0)

In [9]:
# Related orbital elements are:

#    w1 = N + w   = longitude of perihelion
#    L  = M + w1  = mean longitude
#    q  = a*(1-e) = perihelion distance
#    Q  = a*(1+e) = aphelion distance
#    P  = a ^ 1.5 = orbital period (years if a is in AU, astronomical units)
#    T  = Epoch_of_M - (M(deg)/360_deg) / P  = time of perihelion
#    v  = true anomaly (angle between position and perihelion)
#    EA  = eccentric anomaly

# The position of the Sun:

# Eccentric Anomaly:
EA_s = M_s + e_s * sin(M_s) * ( 1.0 + e_s * cos(M_s) )

# The Sun's distance and true anomaly:

xv_s = a_s * ( cos(EA_s) - e_s )
yv_s = a_s * ( sqrt(1.0 - e_s*e_s) * sin(EA_s) )

v_s = atan2( yv_s, xv_s )
r_s = sqrt( xv_s*xv_s + yv_s*yv_s )

L_s = v_s + w_s

# The position of the Moon:

# Eccentric Anomaly:
EA_m = M_m + e_m * sin(M_m) * ( 1.0 + e_m * cos(M_m) )

# The Moon's distance and true anomaly:

xv_m = a_m * ( cos(EA_m) - e_m )
yv_m = a_m * ( sqrt(1.0 - e_m*e_m) * sin(EA_m) )

v_m = atan2( yv_m, xv_m )
r_m = sqrt( xv_m*xv_m + yv_m*yv_m )

L_m = v_m + w_m

In [10]:
# Compute Sidereal Time

# The Local Sideral Time (LST) is simply the RA of your local meridian. 
# The Greenwich Mean Sideral Time (GMST) is the LST at Greenwich. 
# The Greenwich Mean Sidereal Time at 0h UT (GMST0) is the GMST at Greenwich midnight. 

# GMST0 is computed from L_s (divide Ls by 15 to get GMST0 in hours rather than degrees).
# Get GMST by adding the UT hours of the day, and get LST by adding your local longitude.
# From the Prime Meridian at Greenwich: east longitude is positive, west is negative.

# Assume times (UT) are in hours and angles (Ls, local longitude) in degrees, 
# and use the conversion factor of 15 degrees/hour:

GMST0 = L_s/15.0 + 12.0
GMST = GMST0 + UT
LST  = GMST + loclon/15.0

In [11]:
# The position of the Moon in cartesian and geocentric ecliptic coordinates:

xg_m = r_m * ( cos(N_m) * cos(L_m) - sin(N_m) * sin(L_m) * cos(i_m) )
yg_m = r_m * ( sin(N_m) * cos(L_m) + cos(N_m) * sin(L_m) * cos(i_m) )
zg_m = r_m * ( sin(L_m) * sin(i_m) )

lon_m = atan2( yg_m, xg_m )
lat_m = atan2( zg_m, sqrt(xg_m*xg_m+yg_m*yg_m) )

In [12]:
# Precession

# As the Earth's equatorial plane precesses relative to the ecliptic plane, the equinoxes 
# move westward along the ecliptic at 50.3 arcseconds per year = 3.82E-5 degrees per day. 

# Precession is needed to compute the Moon's position for some standard epoch, such as 2000.0 
# (e.g. to be able to plot the position on a star atlas). 

# Precession correction is not needed for the position for today's epoch 
# (e.g. when computing rising/setting times).

# Add the correction below to the ecliptic longitude (degrees).

Epoch = 2000.0
dlon_m = 3.82394E-5 * ( 365.2422 * ( Epoch - 2000.0 ) - d0 )

In [13]:
# Perturbations of the Moon

# If the position of the Moon is needed with a better accuracy than about 2 degrees, 
# then the most important perturbations must be taken into account. 
# For 2 arc minute accuracy, all the following terms should be accounted for. 
# If less accuracy is needed, some of the smaller terms can be omitted.

# First compute:

L_s = M_s + w_s           # Mean Longitude of the Sun  (N_sun=0)
L_m = M_m + w_m + N_m     # Mean longitude of the Moon
D_m = L_m - L_s           # Mean elongation of the Moon
F_m = L_m - N_m           # Argument of latitude for the Moon

# Add these terms to the Moon's geocentric ecliptic longitude (scale factors are for degrees):

dlon_m += -1.274 * sin(M_m - 2*D_m) # the Evection
dlon_m += +0.658 * sin(2*D_m)      # the Variation
dlon_m += -0.186 * sin(M_s)        # the Yearly Equation
dlon_m += -0.059 * sin(2*M_m - 2*D_m)
dlon_m += -0.057 * sin(M_m - 2*D_m + M_s)
dlon_m += +0.053 * sin(M_m + 2*D_m)
dlon_m += +0.046 * sin(2*D_m - M_s)
dlon_m += +0.041 * sin(M_m - M_s)
dlon_m += -0.035 * sin(D_m)        # the Parallactic Equation
dlon_m += -0.031 * sin(M_m + M_s)
dlon_m += -0.015 * sin(2*F_m - 2*D_m)
dlon_m += +0.011 * sin(M_m - 4*D_m)
lon_m += radians(dlon_m)

# Add these terms to the Moon's geocentric ecliptic latitude (scale factors are for degrees):

dlat_m = -0.173 * sin(F_m - 2*D_m)
dlat_m += -0.055 * sin(M_m - F_m - 2*D_m)
dlat_m += -0.046 * sin(M_m + F_m - 2*D_m)
dlat_m += +0.033 * sin(F_m + 2*D_m)
dlat_m += +0.017 * sin(2*M_m + F_m)
lat_m += radians(dlat_m)

# Add these terms to the Moon's distance (scale factors are for Earth radii):

dr_m = -0.58 * cos(M_m - 2*D_m)
dr_m += -0.46 * cos(2*D_m)
r_m += dr_m

In [14]:
# the position of the Moon in Geocentric Cartesian ecliptic coordinates

# Now we have the geocentric coordinates of the Moon including perturbations, 
# convert the perturbed ecliptic longitude, latitude, r to (perturbed) x, y, z:

xg_m = r_m * cos(lon_m) * cos(lat_m)
yg_m = r_m * sin(lon_m) * cos(lat_m)
zg_m = r_m              * sin(lat_m)

In [15]:
# Equatorial coordinates

# convert rectangular, ecliptic coordinates to rectangular, equatorial coordinates: 
# simply rotate the y-z-plane by ecl, the angle of the obliquity of the ecliptic:

# the obliquity of the ecliptic (degrees), which is slowly decreasing with time:

ecl = 23.4393 - 3.563E-7 * d0

xe_m = xg_m
ye_m = yg_m * cos(ecl) - zg_m * sin(ecl)
ze_m = yg_m * sin(ecl) + zg_m * cos(ecl)

# Finally, compute the Moon's Right Ascension (RA) and Declination (Dec):

RA_m  = atan2( ye_m, xe_m )
Dec_m = atan2( ze_m, sqrt(xe_m*xe_m+ye_m*ye_m) )

# Re-compute the Moon's geocentric distance:

# rg_m = r + m = sqrt(xg_m*xg_m + yg_m*yg_m + zg_m*zg-m) = sqrt(xe_m*xe_m+ye_m*ye_m+ze_m*ze_m)

In [23]:
# Azimuthal coordinates

# To find the azimuthal coordinates (azimuth and altitude), proceed by computing 
# the HA (Hour Angle) of the object. But first we must compute the LST (Local Sidereal Time), 
# which we do as described in 5b above. When we know LST, we can easily compute HA from:

HA_m = LST - RA_m

# HA is usually given in the interval -12 to +12 hours, or -180 to +180 degrees. If HA is zero, the object can be seen directly to the south. If HA is negative, the object is to the east of south, and if HA is positive, the object is to the west of south. IF your computed HA should fall outside this interval, add or subtract 24 hours (or 360 degrees) until HA falls within this interval.

# convert HA and Decl to local azimuth and altitude. Need local latitude "loclat".
# Azimuth is 0 at North, 90 deg at East, 180 deg at South and 270 deg at West. 
# Altitude is 0 at the (mathematical) horizon, 90 deg at zenith, and negative below the horizon.

x = cos(HA_m) * cos(Dec_m)
y = sin(HA_m) * cos(Dec_m)
z = sin(Dec_m)

loclatr = radians(loclat)
xhor = x * sin(loclatr) - z * cos(loclatr)
yhor = y
zhor = x * cos(loclatr) + z * sin(loclatr)

az  = atan2( yhor, xhor ) + radians(180.0)
alt_geocen = asin( zhor ) # = atan2( zhor, sqrt(xhor*xhor+yhor*yhor) )

In [25]:
# The Moon's topocentric position.

# The Moon's position computed earlier is geocentric, i.e. as seen by an imaginary observer 
# at the center of the Earth. Real observers on the surface of the Earth or on a LEO satellite
# will see a different position - the topocentric position. This position can differ by more 
# than one degree from the geocentric position. 
# To compute the topocentric positions, add a correction to the geocentric position.
# NOTE: velocity aberration from a LEO satellite will be less than a couple of arcseconds.
#       See the ipython notebook: "Aberration Angle".

# Start by computing the Moon's parallax, i.e. the apparent size 
# of the (equatorial) radius of the Earth, as seen from the Moon:

mpar = asin( 1/r_m )

# where r_m is the Moon's distance in Earth radii. It's simplest to apply the correction in 
# horizontal coordinates (azimuth and altitude): within our accuracy aim of 1-2 arc minutes, 
# no correction need be applied to the azimuth: only correct the altitude above the horizon:

alt_topocen = alt_geocen - mpar * cos(alt_geocen)

# Sometimes one need to correct for topocentric position directly in equatorial coordinates 
# though, e.g. if one wants to draw on a star map how the Moon passes in front of the Pleiades, 
# as seen from some specific location. 
# Then we need to know the Moon's geocentric Right Ascension and Declination (RA, Decl), 
# the Local Sidereal Time (LST), and our local latitude (lat).

# Our astronomical latitude (loclat) must first be converted to a geocentric latitude (gclat), 
# and distance from the center of the Earth (rho) in Earth equatorial radii. 
# For an approximate topocentric position, pretend that the Earth is a sphere, and use:

#gclatr = loclatr
#rho = 1.0

# To account for the flattening of the Earth, compute:

gclatr = loclatr - radians(0.1924) * sin(2*loclatr)
rho   = 0.99833 + 0.00167 * cos(2*loclatr)

# We also need an auxiliary angle, g:

g = atan( tan(gclatr) / cos(HA_m) )

# Now convert the geocentric Right Ascension and Declination (RA, Dec) 
# to their topocentric values (topRA, topDec).
# For gclat, and thus g, equal to or extremely close to zero, use a simpler equation:

topRA  = RA_m  - mpar * rho * cos(gclatr) * sin(HA_m) / cos(Dec_m)
if gclatr > 0.001:
    topDec = Dec_m - mpar * rho * sin(gclatr) * sin(g - Dec_m) / sin(g)
else:
    topDec = Dec_m - mpar * rho * sin(-Dec_m) * cos(HA_m)

In [27]:
degrees(topRA), degrees(topDec)

(169.2537867290913, 29.912698995435868)